In [1]:
import preprocess as pp
import configs
import persister
import relation
import lda
import vis
import evaluate
from utils import *

from collections import Counter
from pprint import pprint
from functools import reduce

import re
import numpy as np
from nltk.tree import Tree
from nltk import word_tokenize,sent_tokenize

not use stanford CoreNLP client!


[nltk_data] Error loading sentiwordnet: <urlopen error [Errno 111]
[nltk_data]     Connection refused>


In [ ]:
# is_news = 1

# l = 7
# t = 4
# m = "c_v"
# i = 200
# min_df = 1

is_news = 0
l = 6
t = 14
m = "c_v"
i = 200
min_df = 1

# load
if is_news:
    _raw = persister.load_json(configs.RAWNEWS)
    parse = persister.read_parse()
    _input = persister.read_input(configs.NEWSINPUT)
    model_name = configs.NEWSMODEL.format(l, t, m, i, min_df)
    terms, doc_topic, topic_word = persister.read_lda(configs.NEWSLDA.format(model_name))
    model = persister.load_model(model_name)
    vec = persister.load_model(configs.NEWSVEC.format(min_df))
    print("load news")
else:
    _raw = persister.load_json(configs.RAWABSTRACT)
    _input = persister.read_input(configs.ABSTRACTINPUT)
    model_name = configs.ABSTRACTMODEL.format(l, t, m, i, min_df)
    terms, doc_topic, topic_word = persister.read_lda(configs.ABSTRACTLDA.format(model_name))
    parse = persister.read_parse(configs.ABSTRACTPARSE)
    model = persister.load_model(model_name)
    vec = persister.load_model(configs.ABSVEC.format(min_df))
    print("load abs")
tf = vec.fit_transform(_input)
top_terms, top_docs = lda.get_topics(topic_word, terms, doc_topic)
df_top_words, df_top_docs = lda.pd_topics_vis(top_terms, top_docs)

In [ ]:
len(terms)

In [ ]:
distr = lda.get_dominant_topic(doc_topic)
print("docs distribution:",dict(Counter(distr)))

In [ ]:
print("top terms info:")
df_top_words

In [ ]:
print("top docs info:")
df_top_docs

In [ ]:
# vis.pyLDA(topic_word, doc_topic, [len(s) for s in [word_tokenize(corp) for corp in _input]], vec.get_feature_names(), np.array(sum(tf).todense())[0])

In [ ]:
t_idx = 1
for top_doc in top_docs[t_idx]:
    _idx = top_doc[0]
    if type(parse[_idx]) is str:
        print(_idx, "parse", "err")
        continue
    sents = relation.convert_parse2lemma_sents(parse[_idx])
    sort_idxs, importance, counts = relation.extract_important_sents(sents, [terms[x[0]] for x in top_terms[t_idx]], [x[1] for x in top_terms[t_idx]])
    for i in sort_idxs:
        if importance[i]>0:
            print(round(importance[i], 2), counts[i], _idx, i)
            sent_tokens = parse[_idx]["sentences"][i]["tokens"]
            sent_deps = parse[_idx]["sentences"][i]["enhancedPlusPlusDependencies"]
            print(" ".join([s["lemma"] for s in sent_tokens]))
            sents = sent_tokenize(_raw[_idx])
            if len(sents) > i:
                print(sents[i])
            else:
                print("sent token differ")
            triples = relation.extract_triples_from_sent(sent_deps, sent_tokens, True)
            for triple in triples:
                if None in triple:
                    print(triple)
                    continue
                s = [sent_tokens[i]["originalText"] for i in triple[0]]
                p = [sent_tokens[i]["originalText"] for i in triple[1]]
                o = [sent_tokens[i]["originalText"] for i in triple[2]]
                print("{}-{}-{}".format(s,p,o))
            print("="*40)

In [ ]:
extended = relation.extend_lda_results(parse, _input, top_terms, top_docs, terms, "lemma", top_n=-1, score_method="tfidf")
print(len(extended[t_idx]))
for triple in extended[t_idx]:
    print(round(triple[1],2),"{}-[{}]-{}".format(" ".join(triple[0][0])," ".join(triple[0][1])," ".join(triple[0][2])))

In [ ]:
ws = 110
measure = "c_v"
old = lda.get_coherence(tf, terms, topic_word, _input, measure, window_size=ws)
fake_idxs = []
min_top = 999
for e in extended: # topic
    new_word_idxs = []
    for t in e[:20]:# triples
         new_word_idxs.extend(evaluate.filter_triple2term_idx(t[0], vec))
    if min_top > len(set(new_word_idxs)):
        min_top = len(set(new_word_idxs))
    fake_idxs.append(set(new_word_idxs))
fake_topic_word = evaluate.gen_fake_topic_word(topic_word.shape, fake_idxs)
new = lda.get_coherence(tf, terms, fake_topic_word, _input, measure,min_top,window_size=ws)
print(old, new)
print(np.mean(old), np.mean(new))

In [ ]:
print([terms[i] for i in fake_idxs[1]])
print([terms[i[0]] for i in top_terms[1]])

In [ ]:
ps = relation.get_phrases_by_pattern(parse)
sorted(dict(Counter([" ".join(x) for x in ps])).items(), key=lambda x:x[1], reverse=True)